# Handling Missing Values in Python
### Using Titanic Dataset - A Practical Guide

এই নোটবুকে আমরা শিখবো কিভাবে Missing Values হ্যান্ডেল করতে হয়।

**Three Main Techniques:**
1. **Mean Imputation** - যখন ডেটা numerical এবং normally distributed
2. **Median Imputation** - যখন ডেটাতে outliers থাকে
3. **Mode Imputation** - যখন ডেটা categorical

---
## Step 1: Import Library & Load Data

In [1]:
# seaborn হলো visualization library, এর সাথে কিছু built-in dataset আছে
import seaborn as sns

# titanic dataset লোড করছি - এতে ৮৯১ জন যাত্রীর তথ্য আছে
df = sns.load_dataset('titanic')

In [2]:
# প্রথম ৫টা row দেখছি - ডেটা কেমন আছে বুঝতে
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


---
## Step 2: Check Missing Values

In [3]:
# isnull() - প্রতিটা cell এ null আছে কিনা check করে (True/False)
# sum() - কতগুলো True আছে সেটা গুনে দেয়
# মানে প্রতিটা column এ কতগুলো missing value আছে সেটা দেখাচ্ছে

df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

**Output Analysis:**
- `age` → ১৭৭টা missing (numerical data)
- `embarked` → ২টা missing (categorical data - S, C, Q)
- `deck` → ৬৮৮টা missing (অনেক বেশি, এটা নিয়ে কাজ করবো না)

In [4]:
# ডেটাসেটের size দেখছি - (rows, columns)
# ৮৯১ জন যাত্রী, ১৫টা column

df.shape

(891, 15)

---
## Step 3: Why Not Just Drop Missing Values?

In [5]:
# dropna() - যেসব row তে missing value আছে সেগুলো ফেলে দেয়
# দেখো কতটা ডেটা হারাচ্ছি!

df.dropna().shape

(182, 15)

**সমস্যা:** ৮৯১ থেকে মাত্র ১৮২টা row বাকি থাকলো! 

প্রায় **৮০% ডেটা** হারিয়ে গেল। এত ডেটা ফেলে দিলে model এর accuracy খারাপ হবে।

**Solution:** Missing values ফেলে না দিয়ে imputation করবো - মানে ফাঁকা জায়গায় কিছু বসাবো।

---
## Technique 1: Mean Value Imputation

**কখন ব্যবহার করবো?**
- Numerical data
- Data normally distributed (outlier নেই)

**Mean কী?** সব value যোগ করে total count দিয়ে ভাগ।

In [6]:
# age column এর statistics দেখছি
# mean, min, max, std সব একসাথে পাবো

df['age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: age, dtype: float64

In [7]:
# age এর mean বের করছি
# প্রায় ২৯.৭ বছর

mean_age = df['age'].mean()
print(f"Mean Age: {mean_age}")

Mean Age: 29.69911764705882


In [8]:
# fillna() - NaN values কে specified value দিয়ে replace করে
# নতুন column তৈরি করছি mean দিয়ে fill করা

df['age_mean'] = df['age'].fillna(mean_age)

In [9]:
# check করছি - age_mean এ কোনো missing value নেই

df['age_mean'].isnull().sum()

np.int64(0)

---
## Technique 2: Median Value Imputation

**কখন ব্যবহার করবো?**
- Numerical data
- Data তে **outliers আছে** (extreme values)

**Median কী?** সব value sort করলে মাঝখানের value।

**কেন Median ভালো যখন outlier থাকে?**

ধরো ৫ জনের বয়স: `[25, 26, 27, 28, 80]`
- Mean = (25+26+27+28+80)/5 = **37.2** ❌ (বেশিরভাগের কাছে ভুল)
- Median = মাঝখানেরটা = **27** ✅ (সঠিক representation)

In [10]:
# age এর median বের করছি
# ২৮ বছর - mean (২৯.৭) থেকে কম কারণ বয়স্ক মানুষ (outlier) influence করছে না

median_age = df['age'].median()
print(f"Median Age: {median_age}")

Median Age: 28.0


In [11]:
# median দিয়ে fill করছি

df['age_median'] = df['age'].fillna(median_age)

In [12]:
# তিনটা column পাশাপাশি compare করছি
# যেখানে age NaN ছিল সেখানে age_mean এ ২৯.৭ আর age_median এ ২৮ বসেছে

df[['age', 'age_mean', 'age_median']]

,age,age_mean,age_median
0,22.0,22.000000,22.0
1,38.0,38.000000,38.0
2,26.0,26.000000,26.0
3,35.0,35.000000,35.0
4,35.0,35.000000,35.0
...,...,...,...
886,27.0,27.000000,27.0
887,19.0,19.000000,19.0
888,NaN,29.699118,28.0
889,26.0,26.000000,26.0


---
## Technique 3: Mode Imputation (For Categorical Data)

**কখন ব্যবহার করবো?**
- **Categorical data** (text/category type)
- Mean/Median এর কোনো meaning নেই এখানে

**Mode কী?** যে value সবচেয়ে বেশিবার এসেছে।

**Example:** `embarked` column এ S, C, Q আছে। এগুলো port এর নাম:
- S = Southampton
- C = Cherbourg  
- Q = Queenstown

In [13]:
# embarked এ কোন কোন unique value আছে দেখছি
# S, C, Q আর nan (missing)

df['embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [14]:
# কোন row তে embarked missing সেগুলো দেখছি
# ২টা row - index 61 আর 829

df[df['embarked'].isnull()]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_mean,age_median
61,1,1,female,38.0,0,0,80.0,NaN,First,woman,False,B,NaN,yes,True,38.0,38.0
829,1,1,female,62.0,0,0,80.0,NaN,First,woman,False,B,NaN,yes,True,62.0,62.0


In [15]:
# প্রতিটা port থেকে কতজন উঠেছে গুনছি
# S থেকে সবচেয়ে বেশি - এটাই mode

df['embarked'].value_counts()

embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [16]:
# mode বের করছি
# mode() একটা Series return করে, তাই [0] দিয়ে প্রথম value নিচ্ছি

mode_value = df['embarked'].mode()[0]
print(f"Mode (Most Frequent): {mode_value}")

Mode (Most Frequent): S


In [17]:
# mode দিয়ে fill করছি
# ফাঁকা জায়গায় 'S' বসবে

df['embarked_mode'] = df['embarked'].fillna(mode_value)

In [18]:
# verify করছি - এখন আর কোনো missing নেই

df['embarked_mode'].isnull().sum()

np.int64(0)

In [19]:
# compare করছি - original vs filled

df[['embarked', 'embarked_mode']]

,embarked,embarked_mode
0,S,S
1,C,C
2,S,S
3,S,S
4,S,S
...,...,...
886,S,S
887,S,S
888,S,S
889,C,C


---
## Summary: কোন Technique কখন?

| Data Type | Condition | Technique |
|-----------|-----------|----------|
| Numerical | Normal distribution, no outliers | **Mean** |
| Numerical | Outliers present (extreme values) | **Median** |
| Categorical | Text/Category values | **Mode** |

### Key Takeaways:
1. **আগে ডেটা বোঝো** - `describe()`, `unique()`, `value_counts()` দিয়ে
2. **সঠিক technique বেছে নাও** - data type আর distribution অনুযায়ী
3. **Verify করো** - `isnull().sum()` দিয়ে check করো missing handle হয়েছে কিনা